
# DSE ↔ PROD Overlap — **Single Joined CSV (No Joins Needed)**

This notebook expects **one CSV** at `utils/dataset.csv` (already joined) and computes:

- Per‑row **alert flags** for DSE and PROD using per‑channel thresholds.
- **Summaries by channel**: counts, alerts, fraud‑in‑alerts for DSE and PROD.
- **Alert overlap** in the *same row*: intersection, DSE‑only (**PROD false negatives**), PROD‑only (**PROD false positives**), plus percentages.
- **Fraud overlap within alerts** similarly framed.
- **Score diagnostics for mismatches**: distributions and **margins vs thresholds**.
- **Hour‑of‑day** distributions for intersections and mismatches.

> Tip: If your channels/thresholds differ, edit the `CONFIG` cell below.


In [ ]:

# ============================
# CONFIG — paths & thresholds
# ============================
from pathlib import Path

CONFIG = {
    "path": {
        # Primary path. We auto-fallback to `utils/dataset` (no extension) if needed.
        "csv": "utils/dataset.csv"
    },
    # Column names present in the single joined CSV.
    # If any are missing, the notebook will attempt reasonable fallbacks.
    "cols": {
        "lifecycle_id": "lifecycle_id",
        "channel": "channel",                    # if absent, we'll synthesize 'ALL'
        "event_ts": "event_received_at",         # parsed to hour
        "dse_raw_score": "dse_raw_score",
        "dse_mt_score": "dse_mt_score",
        "prod_raw_score": "raw_score",           # aka shadow / prod raw score
        "prod_mt_score": "mt_score",
        # First match found from this list will be used as boolean fraud label
        "fraud_label_candidates": [
            "fraud_label",
            "is_fraud",
            "fraud",
            "fraud_label_true_scored",
            "fraud_label_true",
            "label_fraud"
        ]
    },
    # If your scores are 0–1 instead of 0–1000, enable scaling to 0–1000 thresholds.
    "scaling": {
        "dse_raw_scale_x1000": True,
        "dse_mt_scale_x1000": True,
        "prod_raw_scale_x1000": True,
        "prod_mt_scale_x1000": True
    },
    # Per-channel thresholds (fill with your Looker values as needed).
    # If a row's channel is unseen, we fall back to 'DEFAULT' if present; otherwise the first key.
    "thresholds": {
        "DG":  {"RS": 980, "MT": 765},
        "FD":  {"RS": 980, "MT": 765},
        "CMB": {"RS": 980, "MT": 765},
        "DEFAULT": {"RS": 980, "MT": 765}
    },
    # Output directory for CSV exports
    "out_dir": "overlap_outputs"
}

In [ ]:

# ======================
# Imports & setup (pandas)
# ======================
import os, sys, math
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", 120)
os.makedirs(CONFIG["out_dir"], exist_ok=True)


In [ ]:

# =====================
# Load the single CSV
# =====================
def _resolve_csv(path_str: str) -> Path:
    p = Path(path_str)
    if p.exists():
        return p
    # Fallback: path without extension
    p2 = Path(str(p).rstrip(".csv"))
    if p2.exists():
        return p2
    # Fallback: utils/dataset vs ./dataset
    if not p.exists():
        p3 = Path("utils") / "dataset.csv"
        if p3.exists():
            return p3
    return p

csv_path = _resolve_csv(CONFIG["path"]["csv"])
if not csv_path.exists():
    raise FileNotFoundError(f"CSV not found at '{csv_path}'. Place your joined file at utils/dataset.csv")

df = pd.read_csv(csv_path)

print(f"Loaded {len(df):,} rows from {csv_path}")
print("Columns:", list(df.columns)[:40], ("..." if df.shape[1] > 40 else ""))
df.head(3)

In [ ]:

# ==================================================
# Prep: ensure needed columns, fill fallbacks
# ==================================================
C = CONFIG["cols"]

# Channel
if C["channel"] not in df.columns:
    df["channel"] = "ALL"

# Fraud label: pick first candidate present, coerce to bool
fraud_col = None
for cand in C["fraud_label_candidates"]:
    if cand in df.columns:
        fraud_col = cand
        break
if fraud_col is None:
    df["fraud_label"] = False
    fraud_col = "fraud_label"
else:
    # Robust cast to boolean (treat 1/'true'/'True' as True)
    df["fraud_label"] = df[fraud_col].astype(str).str.lower().isin(["1","true","t","yes","y"])
    fraud_col = "fraud_label"

# Event hour
ts_col = C["event_ts"]
if ts_col in df.columns:
    ts = pd.to_datetime(df[ts_col], errors="coerce", utc=True)
    # local hour isn't known; use UTC hour to keep consistent
    df["hour"] = ts.dt.hour
else:
    df["hour"] = np.nan

# Score scaling helpers
def _scale_if_needed(series: pd.Series, scale_x1000: bool) -> pd.Series:
    if scale_x1000:
        return series.astype(float) * 1000.0
    return series.astype(float)

# Bring/scale score columns with resilience to variants (_x1000, _rounded)
def _pick_score(base_name: str, fallbacks: list[str]):
    for name in [base_name] + fallbacks:
        if name in df.columns:
            return name
    return None

dse_raw_name = _pick_score(C["dse_raw_score"], [f"{C['dse_raw_score']}_x1000", f"{C['dse_raw_score']}_X1000",
                                               f"{C['dse_raw_score']}_x1000_rounded"]) or C["dse_raw_score"]
dse_mt_name  = _pick_score(C["dse_mt_score"],  [f"{C['dse_mt_score']}_x1000", f"{C['dse_mt_score']}_X1000",
                                               f"{C['dse_mt_score']}_x1000_rounded"]) or C["dse_mt_score"]
prod_raw_name = _pick_score(C["prod_raw_score"], [f"{C['prod_raw_score']}_x1000", f"{C['prod_raw_score']}_X1000",
                                                 f"{C['prod_raw_score']}_x1000_rounded", "shadow_ob_score"]) or C["prod_raw_score"]
prod_mt_name  = _pick_score(C["prod_mt_score"],  [f"{C['prod_mt_score']}_x1000", f"{C['prod_mt_score']}_X1000",
                                                 f"{C['prod_mt_score']}_x1000_rounded", "mt_score_prod"]) or C["prod_mt_score"]

# Build scaled numeric columns
df["dse_raw"]  = _scale_if_needed(pd.to_numeric(df.get(dse_raw_name, pd.Series(np.nan, index=df.index)), errors="coerce"),
                                   CONFIG["scaling"]["dse_raw_scale_x1000"]
                                  )
df["dse_mt"]   = _scale_if_needed(pd.to_numeric(df.get(dse_mt_name,  pd.Series(np.nan, index=df.index)), errors="coerce"),
                                   CONFIG["scaling"]["dse_mt_scale_x1000"]
                                  )
df["prod_raw"] = _scale_if_needed(pd.to_numeric(df.get(prod_raw_name, pd.Series(np.nan, index=df.index)), errors="coerce"),
                                   CONFIG["scaling"]["prod_raw_scale_x1000"]
                                  )
df["prod_mt"]  = _scale_if_needed(pd.to_numeric(df.get(prod_mt_name,  pd.Series(np.nan, index=df.index)), errors="coerce"),
                                   CONFIG["scaling"]["prod_mt_scale_x1000"]
                                  )

# Lifecycle ID (string)
lid_col = C["lifecycle_id"] if C["lifecycle_id"] in df.columns else None
if lid_col is None:
    raise KeyError("Expected a 'lifecycle_id' column in the joined dataset.")
df["lifecycle_id"] = df[lid_col].astype(str)


In [ ]:

# ==================================================
# Alert flags & margins vs thresholds (per channel)
# ==================================================
TH = CONFIG["thresholds"]
def _get_th_for_channel(ch: str):
    if ch in TH: 
        return TH[ch]
    if "DEFAULT" in TH:
        return TH["DEFAULT"]
    # fallback to first
    first_key = next(iter(TH))
    return TH[first_key]

def _alert_flags(row):
    th = _get_th_for_channel(row["channel"]) if pd.notna(row["channel"]) else _get_th_for_channel("DEFAULT")
    rs_th, mt_th = th["RS"], th["MT"]
    is_alert_dse  = (pd.notna(row["dse_raw"])  and row["dse_raw"]  >= rs_th) or (pd.notna(row["dse_mt"])  and row["dse_mt"]  >= mt_th)
    is_alert_prod = (pd.notna(row["prod_raw"]) and row["prod_raw"] >= rs_th) or (pd.notna(row["prod_mt"]) and row["prod_mt"] >= mt_th)
    raw_margin_dse  = max((row["dse_raw"]  - rs_th) if pd.notna(row["dse_raw"])  else -np.inf,
                          (row["dse_mt"]   - mt_th) if pd.notna(row["dse_mt"])   else -np.inf)
    raw_margin_prod = max((row["prod_raw"] - rs_th) if pd.notna(row["prod_raw"]) else -np.inf,
                          (row["prod_mt"]  - mt_th) if pd.notna(row["prod_mt"])  else -np.inf)
    return pd.Series({
        "is_alert_dse": bool(is_alert_dse),
        "is_alert_prod": bool(is_alert_prod),
        "margin_dse": raw_margin_dse if np.isfinite(raw_margin_dse) else np.nan,
        "margin_prod": raw_margin_prod if np.isfinite(raw_margin_prod) else np.nan
    })

df = pd.concat([df, df.apply(_alert_flags, axis=1)], axis=1)

# Overlap sets within the same row
df["in_intersection"] = df["is_alert_dse"] & df["is_alert_prod"]
df["dse_only_alert"]  = df["is_alert_dse"] & ~df["is_alert_prod"]
df["prod_only_alert"] = ~df["is_alert_dse"] & df["is_alert_prod"]


In [ ]:

# =============================================
# Summaries by channel (alerts & overlap pct)
# =============================================
def _agg_counts(sub):
    return pd.DataFrame({
        "events": sub.size,
        "alerts_dse": sub["is_alert_dse"].sum(),
        "alerts_prod": sub["is_alert_prod"].sum(),
        "intersect": sub["in_intersection"].sum(),
        "dse_only": sub["dse_only_alert"].sum(),
        "prod_only": sub["prod_only_alert"].sum()
    }, index=[0])

by_channel = df.groupby("channel").apply(_agg_counts).reset_index(level=1, drop=True).reset_index()
by_channel["pct_intersect_of_dse"]  = np.where(by_channel["alerts_dse"]>0, 100*by_channel["intersect"]/by_channel["alerts_dse"], np.nan)
by_channel["pct_intersect_of_prod"] = np.where(by_channel["alerts_prod"]>0, 100*by_channel["intersect"]/by_channel["alerts_prod"], np.nan)

display(by_channel)
by_channel.to_csv(Path(CONFIG["out_dir"]) / "summary_by_channel.csv", index=False)


In [ ]:

# =======================================================
# Fraud‑in‑alerts overlap (within alerts only)
# =======================================================
mask_alerts = df["is_alert_dse"] | df["is_alert_prod"]
alerts = df.loc[mask_alerts].copy()
alerts["fraud_in_dse_alert"]  = alerts["is_alert_dse"]  & alerts["fraud_label"]
alerts["fraud_in_prod_alert"] = alerts["is_alert_prod"] & alerts["fraud_label"]
alerts["fraud_intersection"]  = alerts["in_intersection"] & alerts["fraud_label"]
alerts["fraud_dse_only"]      = alerts["dse_only_alert"] & alerts["fraud_label"]
alerts["fraud_prod_only"]     = alerts["prod_only_alert"] & alerts["fraud_label"]

fraud_by_channel = alerts.groupby("channel").agg({
    "fraud_in_dse_alert": "sum",
    "fraud_in_prod_alert": "sum",
    "fraud_intersection": "sum",
    "fraud_dse_only": "sum",
    "fraud_prod_only": "sum",
}).reset_index()

display(fraud_by_channel)
fraud_by_channel.to_csv(Path(CONFIG["out_dir"]) / "fraud_overlap_by_channel.csv", index=False)


In [ ]:

# ==========================================================
# Margin diagnostics for mismatches (how far from thresholds)
# ==========================================================
dse_only = df.loc[df["dse_only_alert"]].copy()
prod_only = df.loc[df["prod_only_alert"]].copy()

def _summarise_margins(sub, margin_col):
    return sub.groupby("channel")[margin_col].agg(["count","mean","min","max"]).reset_index()

dse_margins = _summarise_margins(dse_only, "margin_dse")
prod_margins = _summarise_margins(prod_only, "margin_prod")

display(dse_margins)
display(prod_margins)

dse_margins.to_csv(Path(CONFIG["out_dir"]) / "dse_only_margin_summary.csv", index=False)
prod_margins.to_csv(Path(CONFIG["out_dir"]) / "prod_only_margin_summary.csv", index=False)


In [ ]:

# ==================================
# Hour‑of‑day distributions (UTC)
# ==================================
def _by_hour(df_in, flag_col, hour_col="hour"):
    t = df_in.loc[df_in[flag_col]].copy()
    if hour_col not in t.columns:
        return pd.DataFrame(columns=["hour","count"]).assign(hour=pd.Series(dtype=float), count=pd.Series(dtype=int))
    out = t.groupby(hour_col).size().reset_index(name="count").sort_values("hour")
    return out

inter_by_hour = _by_hour(df, "in_intersection")
dse_only_by_hour = _by_hour(df, "dse_only_alert")
prod_only_by_hour = _by_hour(df, "prod_only_alert")

display(inter_by_hour)
display(dse_only_by_hour)
display(prod_only_by_hour)

inter_by_hour.to_csv(Path(CONFIG["out_dir"]) / "intersection_by_hour.csv", index=False)
dse_only_by_hour.to_csv(Path(CONFIG["out_dir"]) / "dse_only_by_hour.csv", index=False)
prod_only_by_hour.to_csv(Path(CONFIG["out_dir"]) / "prod_only_by_hour.csv", index=False)

# Fraud‑in‑alerts hours
fraud_inter_by_hour = _by_hour(alerts, "fraud_intersection")
fraud_dse_only_by_hour = _by_hour(alerts, "fraud_dse_only")
fraud_prod_only_by_hour = _by_hour(alerts, "fraud_prod_only")

display(fraud_inter_by_hour)
display(fraud_dse_only_by_hour)
display(fraud_prod_only_by_hour)

fraud_inter_by_hour.to_csv(Path(CONFIG["out_dir"]) / "fraud_intersection_by_hour.csv", index=False)
fraud_dse_only_by_hour.to_csv(Path(CONFIG["out_dir"]) / "fraud_dse_only_by_hour.csv", index=False)
fraud_prod_only_by_hour.to_csv(Path(CONFIG["out_dir"]) / "fraud_prod_only_by_hour.csv", index=False)
